In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'


engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_prices_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [2]:
house_prices_df = pd.concat([house_prices_df,pd.get_dummies(house_prices_df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
house_prices_df = pd.concat([house_prices_df,pd.get_dummies(house_prices_df.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(house_prices_df.mszoning, prefix="mszoning", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(house_prices_df.street, prefix="street", drop_first=True).columns)

In [3]:
X = house_prices_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf'] + dummy_column_names]
Y = house_prices_df.saleprice

In [4]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     482.0
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        22:50:36   Log-Likelihood:                -17475.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1449   BIC:                         3.503e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.173e+05    1.8e+04     -6.502      0.000   -1.53e+05   -8.19e+04
overallqual  2.333e+04   1088.506     21.430      0.000    2.12e+04    2.55e+04
grlivarea      45.6344      2.468     18.494      0.000      40.794      50.475
garagecars   1.345e+04   2990.453      4.498      0.000    7584.056    1.93e+04
garagearea     16.4082     10.402      1.577      0.115      -3.997      36.813
totalbsmtsf    28.3816      2.931      9.684      0.000      22.633      34.131
mszoning_FV  2.509e+04   1.37e+04      1.833      0.067   -1761.679    5.19e+04
mszoning_RH  1.342e+04   1.58e+04      0.847      0.397   -1.77e+04    4.45e+04
mszoning_RL  2.857e+04   1.27e+04      2.246      0.025    3612.782    5.35e+04
mszoning_RM  9047.7894   1.29e+04      0.702      0.483   -1.62e+04    3.43e+04
street_Pave   151.0376   1.63e+04      0.009      0.993   -3.18e+04    3.21e+04
==============================================================================
Omnibus:                      415.883   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            41281.526
Skew:                          -0.115   Prob(JB):                         0.00
Kurtosis:                      29.049   Cond. No.                     5.53e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.53e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

R-Square and Adjusted R-Square looks goof with some room for improvement (0.77).

F statistic is 482.0, and the associated p-value is very close to zero. This means that the features add some information to the reduced model and this model is useful in explaining the target variable.

AIC = 34,970 , BIC = 35,030. These statistics does not provide usueful information on its own. It can be used to compare this model to other models.

The model is satisfactory but there is a room for improvement. In order to imporve the model, two features will be added:

- First Feature : Interaction between grlivarea and garagearea features
- Second Feature : Interaction between grlivarea and overallqual features

In [9]:
house_prices_df['grlivarea_garagearea'] = house_prices_df['grlivarea'] * house_prices_df['garagearea']
house_prices_df['grlivarea_overallqual'] = house_prices_df['grlivarea'] * house_prices_df['overallqual']

In [10]:
X2 = house_prices_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'grlivarea_garagearea', 
                     'grlivarea_overallqual'] + dummy_column_names]

In [11]:
X2 = sm.add_constant(X2)

results = sm.OLS(Y, X2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     460.8
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        23:09:14   Log-Likelihood:                -17396.
No. Observations:                1460   AIC:                         3.482e+04
Df Residuals:                    1447   BIC:                         3.489e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -6561.3553   1.96e+04     -0.334      0.738   -4.51e+04    3.19e+04
overallqual           -3632.2407   2367.768     -1.534      0.125   -8276.866    1012.384
grlivarea               -40.6093      7.508     -5.408      0.000     -55.338     -25.881
garagecars             1.483e+04   2909.754      5.098      0.000    9126.452    2.05e+04
garagearea              103.5538     17.593      5.886      0.000      69.044     138.064
totalbsmtsf              25.7803      2.823      9.132      0.000      20.243      31.318
grlivarea_garagearea     -0.0617      0.009     -7.012      0.000      -0.079      -0.044
grlivarea_overallqual    18.0040      1.423     12.654      0.000      15.213      20.795
mszoning_FV            4.329e+04   1.31e+04      3.316      0.001    1.77e+04    6.89e+04
mszoning_RH            2.888e+04   1.51e+04      1.916      0.056    -685.899    5.84e+04
mszoning_RL            4.361e+04   1.21e+04      3.599      0.000    1.98e+04    6.74e+04
mszoning_RM            2.343e+04   1.23e+04      1.908      0.057    -663.267    4.75e+04
street_Pave            1921.9986   1.54e+04      0.124      0.901   -2.84e+04    3.22e+04
==============================================================================
Omnibus:                      807.080   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            84905.376
Skew:                          -1.625   Prob(JB):                         0.00
Kurtosis:                      40.217   Cond. No.                     2.65e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.65e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The model has imporoved, R-square increased from 0.77 to 0.79. It is not a big increase but it is an improvement . The increase was in both R-Square and Adjusted R-square as well.

AIC and BIC has decreased a little and F-statistic has decreased. These values does not show a significant improvement in the model. For the next model, the target value will be log transfored

In [12]:
Y2 = np.log1p(house_prices_df['saleprice'])

In [13]:
X2 = sm.add_constant(X2)

results = sm.OLS(Y2, X2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     613.5
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        23:14:07   Log-Likelihood:                 587.21
No. Observations:                1460   AIC:                            -1148.
Df Residuals:                    1447   BIC:                            -1080.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     9.9624      0.088    113.401      0.000       9.790      10.135
overallqual               0.1112      0.011     10.491      0.000       0.090       0.132
grlivarea                 0.0003   3.36e-05      9.943      0.000       0.000       0.000
garagecars                0.0702      0.013      5.391      0.000       0.045       0.096
garagearea                0.0006   7.87e-05      7.361      0.000       0.000       0.001
totalbsmtsf               0.0001   1.26e-05     10.962      0.000       0.000       0.000
grlivarea_garagearea  -3.018e-07   3.94e-08     -7.664      0.000   -3.79e-07   -2.25e-07
grlivarea_overallqual  6.135e-06   6.37e-06      0.964      0.335   -6.35e-06    1.86e-05
mszoning_FV               0.5470      0.058      9.364      0.000       0.432       0.662
mszoning_RH               0.4113      0.067      6.098      0.000       0.279       0.544
mszoning_RL               0.5332      0.054      9.833      0.000       0.427       0.640
mszoning_RM               0.3762      0.055      6.845      0.000       0.268       0.484
street_Pave               0.0003      0.069      0.004      0.996      -0.135       0.136
==============================================================================
Omnibus:                      379.074   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2673.767
Skew:                          -1.019   Prob(JB):                         0.00
Kurtosis:                       9.309   Cond. No.                     2.65e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.65e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

After log transforming the target variable, R-Square and adjusted R-Square become better 0.836 and 0.834 respectivly. F-Statistic is the higher over all previous models, and AIC and BIC statistics are much lower (-1,148 , -1,080)

The last model is the best model among all the models.